In [5]:
# -*- coding: utf-8 -*-
"""
ЯЧЕЙКА 1: ПОДГОТОВКА ОКРУЖЕНИЯ И ДАТАСЕТА
"""
from google.colab import drive
import os
import zipfile

# Монтируем диск
drive.mount('/content/drive')

print("\n📦 Устанавливаю зависимости...")
!pip install -q insightface onnxruntime-gpu opencv-python
print("✅ Окружение готово!\n")

# Настройки путей
ZIP_PATH = '/content/drive/MyDrive/students_prepared_dataset.zip'
EXTRACT_PATH = '/content/students_data'
MODEL_PATH = '/content/drive/MyDrive/student_embeddings.pkl'

# Очистка и распаковка
if not os.path.exists(EXTRACT_PATH):
    print("📦 Распаковка датасета...")
    os.makedirs(EXTRACT_PATH, exist_ok=True)
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(EXTRACT_PATH)
    print("✅ Распаковка завершена.")
else:
    print("✅ Датасет уже распакован.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

📦 Устанавливаю зависимости...
✅ Окружение готово!

📦 Распаковка датасета...
✅ Распаковка завершена.


In [6]:
"""
ЯЧЕЙКА 2: ИНИЦИАЛИЗАЦИЯ ИНТЕЛЛЕКТА
"""
import cv2
import numpy as np
from insightface.app import FaceAnalysis

print("🚀 Загрузка модели InsightFace (buffalo_l)...")
# Используем размер 640x640, как в твоем скрипте, для баланса скорости и точности
app = FaceAnalysis(name='buffalo_l', providers=['CUDAExecutionProvider'])
app.prepare(ctx_id=0, det_size=(640, 640))
print("✅ Модель готова к работе на GPU!")

🚀 Загрузка модели InsightFace (buffalo_l)...
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'sdpa_kernel': '0', 'use_tf32': '1', 'fuse_conv_bias': '0', 'prefer_nhwc': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_enable': '0', 'use_ep_level_unified_stream': '0', 'device_id': '0', 'has_user_compute_stream': '0', 'gpu_external_empty_cache': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_external_free': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'enable_cuda_graph': '0', 'user_compute_stream': '0', 'cudnn_conv_use_max_workspace': '1'}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecut

In [7]:
"""
ЯЧЕЙКА 3: ИЗВЛЕЧЕНИЕ ПРИЗНАКОВ
"""
from tqdm import tqdm

def extract_raw_embeddings(data_dir):
    raw_db = {}
    students = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]

    print(f"👥 Найдено студентов для обработки: {len(students)}")

    for student in tqdm(students, desc="Извлечение признаков"):
        student_dir = os.path.join(data_dir, student)
        embeddings = []

        for img_name in os.listdir(student_dir):
            if not img_name.lower().endswith(('.jpg', '.png', '.jpeg')):
                continue

            img_path = os.path.join(student_dir, img_name)
            img = cv2.imread(img_path)
            if img is None: continue

            faces = app.get(img)
            if not faces: continue

            # Логика из твоего скрипта: берем самое крупное лицо
            main_face = max(faces, key=lambda f: (f.bbox[2]-f.bbox[0]) * (f.bbox[3]-f.bbox[1]))

            # Сохраняем уже нормализованный вектор
            embeddings.append(main_face.normed_embedding)

        if embeddings:
            raw_db[student] = np.array(embeddings)

    return raw_db

raw_database = extract_raw_embeddings(EXTRACT_PATH)

👥 Найдено студентов для обработки: 2


Извлечение признаков: 100%|██████████| 2/2 [00:06<00:00,  3.19s/it]


In [8]:
"""
ЯЧЕЙКА 4: ФОРМИРОВАНИЕ ЗОЛОТОЙ БАЗЫ И СОХРАНЕНИЕ
"""
import pickle

def create_gold_embeddings(raw_embeddings_dict, percentile=75):
    gold_embeddings = {}

    print("✨ Очистка от шума и создание эталонов...")
    for person_name, embeddings in raw_embeddings_dict.items():
        if len(embeddings) < 3:
            # Если мало данных - просто усредняем
            centroid = np.mean(embeddings, axis=0)
            gold_embeddings[person_name] = centroid / np.linalg.norm(centroid)
            continue

        # 1. Черновой центроид
        initial_centroid = np.mean(embeddings, axis=0)
        initial_centroid = initial_centroid / np.linalg.norm(initial_centroid)

        # 2. Считаем косинусное сходство через скалярное произведение (векторы нормализованы)
        # sim = e * g
        similarities = np.dot(embeddings, initial_centroid)

        # 3. Оставляем только те кадры, которые лучше порога (отсекаем худшие 25%)
        threshold = np.percentile(similarities, 100 - percentile)
        filtered_embeddings = embeddings[similarities >= threshold]

        # 4. Чистовой центроид
        final_centroid = np.mean(filtered_embeddings, axis=0)
        gold_embeddings[person_name] = final_centroid / np.linalg.norm(final_centroid)

    return gold_embeddings

# Применяем фильтр
gold_database = create_gold_embeddings(raw_database, percentile=75)

# Сохраняем "мозг"
with open(MODEL_PATH, 'wb') as f:
    pickle.dump(gold_database, f)

print(f"🎯 Готово! База золотых эмбеддингов сохранена: {MODEL_PATH}")

✨ Очистка от шума и создание эталонов...
🎯 Готово! База золотых эмбеддингов сохранена: /content/drive/MyDrive/student_embeddings.pkl
